## Web Scraping de Discursos de Casa Rosada

Este cuaderno contiene un script de Python diseñado para realizar web scraping en el sitio web de la Casa Rosada y extraer los discursos públicos del Presidente. El objetivo principal es recolectar información relevante de estos discursos para su posterior análisis.

### Funcionalidades Clave:

1.  **Extracción de Lista de Discursos**: Navega a la sección de "Discursos" de la Casa Rosada y extrae los enlaces y títulos de los discursos más recientes.
2.  **Extracción de Contenido Individual**: Para cada discurso identificado, el script accede a su página individual para extraer el título, la fecha de publicación, el contenido completo (crudo) y realiza una limpieza del texto para eliminar ruido y formatearlo adecuadamente.
3.  **Procesamiento y Estadísticas**: Calcula estadísticas básicas para cada discurso, como la longitud del texto (caracteres y palabras) y el número de párrafos.
4.  **Guardado de Resultados**: Al finalizar el proceso, los datos recolectados son guardados en dos formatos: un archivo CSV y un archivo JSON, lo que facilita su uso en diferentes herramientas de análisis.
5.  **Resumen y Estadísticas Finales**: Muestra un resumen detallado de cada discurso extraído y proporciona estadísticas globales del conjunto de datos.

Este script es una herramienta útil para investigadores, periodistas o cualquier persona interesada en el análisis de los discursos presidenciales argentinos.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Define the base path for saving files in Google Drive
DRIVE_FOLDER_PATH = '/content/drive/MyDrive/nlp/txt para rag'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# @title
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime
import time
import os

class ScraperDiscursosCasaRosada:
    def __init__(self):
        self.base_url = "https://www.casarosada.gob.ar/informacion/discursos"
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        self.discursos = []

    def extraer_lista_discursos(self):
        """
        Extrae la lista de los últimos discursos de la página principal
        """
        try:
            print(f"🌐 Conectando a: {self.base_url}")
            response = requests.get(self.base_url, headers=self.headers, timeout=10)
            response.raise_for_status()

            soup = BeautifulSoup(response.content, 'html.parser')
            print("✅ Página cargada correctamente")

            # Estrategia más simple: buscar todos los enlaces y filtrar inteligentemente
            todos_los_enlaces = soup.find_all('a', href=True)
            print(f"🔍 Encontrados {len(todos_los_enlaces)} enlaces en total")

            discursos_links = []

            for enlace in todos_los_enlaces:
                texto = enlace.get_text(strip=True)
                href = enlace['href']

                # Filtrar enlaces que NO queremos
                if (not texto or
                    len(texto) < 10 or  # Reducido de 15 a 10 para más flexibilidad
                    texto.lower() in ['discursos', 'inicio', 'volver', 'volver al inicio', 'ir al contenido'] or
                    '#' in href or  # Enlaces internos
                    'javascript:' in href.lower()):
                    continue

                # Construir URL completa
                if href.startswith('/'):
                    url_completa = f"https://www.casarosada.gob.ar{href}"
                elif href.startswith('http'):
                    url_completa = href
                else:
                    url_completa = f"https://www.casarosada.gob.ar/{href}"

                # Incluir enlaces que parezcan ser discursos individuales
                # Ser más permisivo con los criterios
                es_posible_discurso = (
                    # Criterio 1: Contiene palabras clave
                    any(palabra in texto.lower() for palabra in [
                        'discurso', 'milei', 'presidente', 'palabras', 'alocución',
                        'mensaje', 'javier', 'conferencia', 'declaración', 'ministro',
                        'argentina', 'nación', 'gobierno', 'política', 'economía'
                    ]) or
                    # Criterio 2: Tiene formato de fecha + título
                    (re.search(r'\d{1,2}\s+de\s+\w+', texto) and len(texto) > 20) or  # Reducido de 30 a 20
                    # Criterio 3: URL contiene patrones de discursos
                    any(patron in url_completa.lower() for patron in [
                        '/discursos/', '/discurso', 'noticia', 'article', 'informacion'
                    ]) or
                    # Criterio 4: Texto suficientemente largo y parece contenido
                    (len(texto) > 25 and any(caracter in texto for caracter in [':', '-', '–']))
                )

                if es_posible_discurso:
                    discursos_links.append({
                        'titulo': texto,
                        'url': url_completa,
                        'fecha_extraccion': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    })

            # Eliminar duplicados por URL
            discursos_unicos = []
            urls_vistas = set()

            for discurso in discursos_links:
                # Normalizar URL para comparación
                url_normalizada = discurso['url'].split('?')[0]  # Remover parámetros
                if url_normalizada not in urls_vistas:
                    discursos_unicos.append(discurso)
                    urls_vistas.add(url_normalizada)

            print(f"📋 Filtrados {len(discursos_unicos)} discursos potenciales")

            # Ordenar por relevancia (los que tienen fechas primero)
            def relevancia(titulo):
                puntaje = 0
                if re.search(r'\d{1,2}\s+de\s+\w+\s+de\s+\d{4}', titulo):
                    puntaje += 3
                if 'discurso' in titulo.lower():
                    puntaje += 2
                if 'milei' in titulo.lower():
                    puntaje += 2
                if 'presidente' in titulo.lower():
                    puntaje += 1
                if len(titulo) > 50:  # Títulos más largos suelen ser más específicos
                    puntaje += 1
                return puntaje

            discursos_unicos.sort(key=lambda x: relevancia(x['titulo']), reverse=True)

            # Retornar todos los discursos, sin límite de 10
            return discursos_unicos

        except Exception as e:
            print(f"❌ Error al extraer lista de discursos: {e}")
            return []

    def extraer_contenido_discurso(self, url_discurso):
        """
        Extrae el contenido completo de un discurso individual
        """
        try:
            print(f"   📖 Extrayendo contenido de: {url_discurso}")
            response = requests.get(url_discurso, headers=self.headers, timeout=10)
            response.raise_for_status()

            soup = BeautifulSoup(response.content, 'html.parser')

            # Extraer título - buscar en diferentes ubicaciones
            titulo_selectores = [
                'h1',
                '.titulo',
                '.title',
                'header h1',
                'main h1',
                'article h1',
                '.entry-title',
                '.post-title',
                '.noticia-titulo',
                '.page-title',
                '.node-title'
            ]

            titulo = None
            for selector in titulo_selectores:
                titulo_elem = soup.select_one(selector)
                if titulo_elem:
                    titulo = titulo_elem
                    break

            titulo_texto = titulo.get_text(strip=True) if titulo else "Sin título"

            # Estrategias para encontrar el contenido principal
            selectores_contenido = [
                'article .contenido',
                'article .content',
                'article .texto',
                '.nota-contenido',
                '.entry-content',
                '.post-content',
                '.article-content',
                '.field-body',
                '.field-content',
                'main article',
                'article',
                '.contenido',
                '.content',
                '.texto',
                'main',
                '.node-content'
            ]

            contenido = None
            for selector in selectores_contenido:
                contenido_elem = soup.select_one(selector)
                if contenido_elem:
                    contenido = contenido_elem
                    break

            # Si no encontramos con selectores específicos, buscar el área principal de texto
            if not contenido:
                # Buscar el elemento que contiene la mayor cantidad de texto
                elementos_texto = soup.find_all(['div', 'section', 'article'])
                if elementos_texto:
                    # Filtrar elementos con suficiente texto
                    elementos_con_texto = [elem for elem in elementos_texto if len(elem.get_text(strip=True)) > 200]
                    if elementos_con_texto:
                        contenido = max(elementos_con_texto, key=lambda x: len(x.get_text()))

            texto_completo = contenido.get_text(separator='\n', strip=True) if contenido else ""

            # Extraer fecha - buscar en diferentes formatos
            fecha_selectores = [
                'time',
                '.fecha',
                '.date',
                '.fecha-publicacion',
                '.entry-date',
                '.post-date',
                'span.fecha',
                '.fecha-noticia',
                '.created',
                '.publish-date'
            ]

            fecha_element = None
            for selector in fecha_selectores:
                fecha_elem = soup.select_one(selector)
                if fecha_elem:
                    fecha_element = fecha_elem
                    break

            # Si no encontramos fecha con selectores, buscar en el texto
            if not fecha_element:
                # Buscar patrones de fecha en el contenido
                fecha_pattern = re.search(r'\d{1,2}\s+de\s+\w+\s+de\s+\d{4}', texto_completo)
                if fecha_pattern:
                    fecha = fecha_pattern.group()
                else:
                    fecha = "Fecha no disponible"
            else:
                fecha = fecha_element.get_text(strip=True)

            return {
                'titulo': titulo_texto,
                'contenido_crudo': texto_completo,
                'fecha_publicacion': fecha,
                'url': url_discurso
            }

        except Exception as e:
            print(f"   ❌ Error al extraer discurso: {e}")
            return None

    def limpiar_texto(self, texto):
        """
        Limpia y preprocesa el texto extraído
        """
        if not texto:
            return ""

        # 1. Eliminar espacios múltiples y normalizar saltos de línea
        texto = re.sub(r'\n+', '\n', texto)
        texto = re.sub(r' +', ' ', texto)

        # 2. Eliminar caracteres especiales no deseados (mantener puntuación básica y acentos)
        texto = re.sub(r'[^\w\s.,;:!?¿¡()\-áéíóúÁÉÍÓÚñÑ@]', '', texto)

        # 3. Normalizar saltos de línea y espacios
        lineas = [linea.strip() for linea in texto.split('\n') if linea.strip()]
        texto_limpio = '\n'.join(lineas)

        # 4. Eliminar líneas muy cortas que puedan ser ruido (pero mantener párrafos válidos)
        lineas_filtradas = []
        for linea in texto_limpio.split('\n'):
            linea_limpia = linea.strip()
            if (
                len(linea_limpia) > 10 or  # Reducido de 15 a 10
                any(caracter in linea_limpia for caracter in ['.', '!', '?', ':', ';']) and len(linea_limpia) > 5
            ):  # Reducido de 8 a 5
                lineas_filtradas.append(linea_limpia)

        return '\n'.join(lineas_filtradas)

    def procesar_discursos(self):
        """
        Proceso completo de ingesta: extracción y limpieza de los discursos
        """
        print("🔍 Extrayendo lista de discursos...")
        lista_discursos = self.extraer_lista_discursos()

        if not lista_discursos:
            print("❌ No se pudieron extraer los discursos desde la página")
            return []

        print(f"📝 Procesando {len(lista_discursos)} discursos...")

        discursos_exitosos = 0
        for i, discurso_info in enumerate(lista_discursos, 1):
            print(f"  [{i}/{len(lista_discursos)}] 📄 Procesando: {discurso_info['titulo'][:80]}...")

            contenido = self.extraer_contenido_discurso(discurso_info['url'])
            if contenido and len(contenido['contenido_crudo']) > 100:  # Solo si tiene contenido válido
                # Aplicar limpieza
                contenido_limpio = self.limpiar_texto(contenido['contenido_crudo'])

                # Solo incluir si el contenido limpio tiene suficiente texto
                if len(contenido_limpio) > 200:
                    discurso_procesado = {
                        **discurso_info,
                        **contenido,
                        'contenido_limpio': contenido_limpio,
                        'longitud_texto': len(contenido_limpio),
                        'num_parrafos': contenido_limpio.count('\n') + 1,
                        'palabras_aprox': len(contenido_limpio.split())
                    }

                    self.discursos.append(discurso_procesado)
                    discursos_exitosos += 1
                    print(f"     ✅ Extraído: {len(contenido_limpio)} caracteres, {discurso_procesado['palabras_aprox']} palabras aprox.")
                else:
                    print(f"     ⚠️  Descartado: contenido muy corto después de limpieza")
            else:
                print(f"     ❌ Falló la extracción o contenido insuficiente")

            time.sleep(1)  # Respetuoso con el servidor

            # Eliminamos la condición de parada para procesar todos los discursos

        print(f"📊 Resumen: {discursos_exitosos}/{len(lista_discursos)} discursos extraídos exitosamente")
        return self.discursos

    def guardar_resultados(self, save_to_drive=False, drive_folder_path=None):
        """
        Guarda los resultados en archivos de texto individuales en Google Drive.
        """
        if not self.discursos:
            print("No hay datos para guardar")
            return []

        if save_to_drive and drive_folder_path:
            # Ensure the drive folder exists
            output_dir = drive_folder_path # Directly use the provided drive_folder_path
            os.makedirs(output_dir, exist_ok=True)
            print(f"💾 Guardando archivos TXT en Google Drive: {output_dir}")

            saved_files = []
            for i, discurso in enumerate(self.discursos):
                # Generate timestamp
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f") # Add microseconds for higher uniqueness

                # Sanitize title for filename
                safe_title = re.sub(r'[^a-zA-Z0-9áéíóúÁÉÍÓÚñÑ\s]', '', discurso['titulo']).strip()
                safe_title = re.sub(r'\s+', '_', safe_title)
                safe_title = safe_title[:50] # Shorten title part to leave room for timestamp

                if not safe_title:
                    safe_title = f"discurso_sin_titulo"

                # Combine timestamp and sanitized title
                filename = os.path.join(output_dir, f"{timestamp}_{safe_title}.txt")

                try:
                    with open(filename, 'w', encoding='utf-8') as f:
                        f.write(f"Título: {discurso['titulo']}\n")
                        f.write(f"Fecha de publicación: {discurso['fecha_publicacion']}\n")
                        f.write(f"URL: {discurso['url']}\n\n")
                        f.write(discurso['contenido_limpio'])
                    print(f"   ✅ Guardado: {filename}")
                    saved_files.append(filename)
                except Exception as e:
                    print(f"   ❌ Error al guardar {filename}: {e}")
            return saved_files
        else:
            print("Modo de guardado no especificado o ruta de Drive no proporcionada.")
            return []

    def mostrar_resumen(self):
        """
        Muestra un resumen de los discursos extraídos
        """
        if not self.discursos:
            print("No hay discursos para mostrar")
            return

        print("\n" + "="*80)
        print(f"📊 RESUMEN DE {len(self.discursos)} DISCURSOS EXTRAÍDOS")
        print("="*80)

        for i, discurso in enumerate(self.discursos, 1):
            print(f"\n🎯 DISCURSO {i}:")
            print(f"   📖 Título: {discurso['titulo']}")
            print(f"   📅 Fecha publicación: {discurso['fecha_publicacion']}")
            print(f"   📊 Estadísticas:")
            print(f"      - Caracteres: {discurso['longitud_texto']:,}")
            print(f"      - Palabras aprox.: {discurso['palabras_aprox']:,}")
            print(f"      - Párrafos: {discurso['num_parrafos']}")
            print(f"   🔗 URL: {discurso['url'][:100]}...")

# EJECUCIÓN PRINCIPAL
print("🚀 INICIANDO SCRAPING DE DISCURSOS DE LA CASA ROSADA (TODOS LOS DISCURSOS DE MILEI)")
print("="*60)

scraper = ScraperDiscursosCasaRosada()

# Ejecutar el scraping
discursos_procesados = scraper.procesar_discursos()

if discursos_procesados:
    print(f"\n🎉 PROCESAMIENTO COMPLETADO!")
    print(f"✅ Se extrajeron y procesaron {len(discursos_procesados)} discursos")

    # Mostrar resumen
    scraper.mostrar_resumen()

    # Guardar resultados como TXT en Google Drive
    print(f"\n💾 GUARDANDO RESULTADOS EN GOOGLE DRIVE...")
    archivos_guardados = scraper.guardar_resultados(save_to_drive=True, drive_folder_path=DRIVE_FOLDER_PATH)

    print(f"\n📁 Archivos generados en Google Drive:")
    if archivos_guardados:
        for archivo in archivos_guardados:
            print(f"   - {archivo}")
    else:
        print("   No se pudieron guardar archivos.")

    # Estadísticas finales
    total_caracteres = sum(d['longitud_texto'] for d in discursos_procesados)
    total_palabras = sum(d['palabras_aprox'] for d in discursos_procesados)

    print(f"\n📈 ESTADÍSTICAS FINALES:")
    print(f"   📚 Total de discursos: {len(discursos_procesados)}")
    print(f"   🔤 Total de caracteres: {total_caracteres:,}")
    print(f"   🗣️  Total de palabras aprox.: {total_palabras:,}")
    # Convertir fechas a formato comparable para rango temporal
    fechas = []
    for d in discursos_procesados:
        try:
            # Intentar parsear con el formato 'Día DD de Mes de AAAA'
            fecha_str = d['fecha_publicacion'].replace(' de ', ' ')
            # Mapear nombres de meses
            meses_map = {
                'enero': '01', 'febrero': '02', 'marzo': '03', 'abril': '04', 'mayo': '05',
                'junio': '06', 'julio': '07', 'agosto': '08', 'septiembre': '09',
                'octubre': '10', 'noviembre': '11', 'diciembre': '12'
            }
            # Eliminar el día de la semana si existe (ej. 'Jueves')
            fecha_parts = fecha_str.split(' ')
            if fecha_parts[0].isalpha() and len(fecha_parts[0]) > 2: # Check if first part is a day name
                fecha_str_clean = ' '.join(fecha_parts[1:])
            else:
                fecha_str_clean = fecha_str

            for es, en in meses_map.items():
                fecha_str_clean = fecha_str_clean.replace(es, en)

            # Convertir a formato DD MM AAAA y luego a datetime object
            day = fecha_str_clean.split(' ')[0]
            month = fecha_str_clean.split(' ')[1]
            year = fecha_str_clean.split(' ')[2]
            formatted_date_str = f"{day.zfill(2)}-{month}-{year}"

            fechas.append(datetime.strptime(formatted_date_str, '%d-%m-%Y'))
        except Exception as e:
            print(f"Advertencia: No se pudo parsear la fecha '{d['fecha_publicacion']}': {e}")

    if fechas:
        min_fecha = min(fechas).strftime('%d de %B de %Y').replace(' 0', ' ')
        max_fecha = max(fechas).strftime('%d de %B de %Y').replace(' 0', ' ')
        print(f"   📅 Rango temporal: {min_fecha} a {max_fecha}")
    else:
        print(f"   📅 Rango temporal: N/A")

else:
    print("❌ No se pudieron extraer discursos")


🚀 INICIANDO SCRAPING DE DISCURSOS DE LA CASA ROSADA (TODOS LOS DISCURSOS DE MILEI)
🔍 Extrayendo lista de discursos...
🌐 Conectando a: https://www.casarosada.gob.ar/informacion/discursos
✅ Página cargada correctamente
🔍 Encontrados 112 enlaces en total
📋 Filtrados 50 discursos potenciales
📝 Procesando 50 discursos...
  [1/50] 📄 Procesando: Jueves 06 de Noviembre de 2025Discurso del Presidente Javier Milei en el America...
   📖 Extrayendo contenido de: https://www.casarosada.gob.ar/informacion/discursos/51109-discurso-del-presidente-javier-milei-en-el-america-business-forum-de-miami
     ✅ Extraído: 23867 caracteres, 4033 palabras aprox.
  [2/50] 📄 Procesando: Jueves 06 de Noviembre de 2025Discurso del Presidente Javier Milei en la Cena de...
   📖 Extrayendo contenido de: https://www.casarosada.gob.ar/informacion/discursos/51111-discurso-del-presidente-javier-milei-en-la-cena-de-gala-del-cpac-en-mar-a-lago-miami
     ✅ Extraído: 8256 caracteres, 1378 palabras aprox.
  [3/50] 📄 Procesando